# Web Requests & APIs (Beginner • 30 minutes)

Welcome! In this quick class you will:
- Use **GET** to fetch JSON from real APIs
- Add **query parameters**
- Send a simple **POST** request
- Loop through **paginated** data
- Do a tiny **mini-project** using SWAPI (Star Wars API)

We keep it **extra simple**: no error handling, just working examples.

## 0) Setup
We only need a few libraries.

In [ ]:
import requests
import pandas as pd
import time
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (8,4)
requests.__version__

## 1) HTTP Basics (very quick)
- **GET** = read data
- **POST** = send data
- **URL** can include **?query=params**
- Responses often come back as **JSON** (looks like Python dict/list)

## 2) GET a JSON object
We'll call a free test API and look at the JSON. Endpoint: `jsonplaceholder.typicode.com`

In [ ]:
url = "https://jsonplaceholder.typicode.com/todos/1"
r = requests.get(url)
todo = r.json()
todo

### GET with query parameters
Use the `params` argument to filter results (the library builds the URL for you).

In [ ]:
url = "https://jsonplaceholder.typicode.com/posts"
params = {"userId": 1}
r = requests.get(url, params=params)
posts = r.json()
len(posts), posts[:2]  # show how many and preview first two

### Put JSON into a table (DataFrame)
Pandas makes it easy to look at structured data.

In [ ]:
df = pd.DataFrame(posts)
df.head()

## 3) POST a JSON payload
We use `httpbin.org/post`, which **echoes** back what we send so we can see it.

In [ ]:
url = "https://httpbin.org/post"
payload = {"name": "Student", "course": "Web Requests & APIs"}
r = requests.post(url, json=payload)
r.json()["json"]  # the server returns the same JSON we sent

## 4) Pagination (multiple pages)
Some APIs return results in **pages**. We'll use `reqres.in` which has a small users dataset.

In [ ]:
all_users = []
page = 1
while True:
    r = requests.get("https://reqres.in/api/users", params={"page": page})
    js = r.json()
    all_users.extend(js["data"])
    if page >= js["total_pages"]:
        break
    page += 1
    time.sleep(0.2)  # tiny pause

pd.DataFrame(all_users)

## 5) Mini-Project: SWAPI (Star Wars API)
Goal: fetch **all people**, make a small table, and a simple count plot.

Endpoint: `https://swapi.dev/api/people/`

In [ ]:
people = []
url = "https://swapi.dev/api/people/"
while url:
    r = requests.get(url)
    js = r.json()
    people.extend(js["results"])
    url = js["next"]
    time.sleep(0.2)

len(people), people[0]["name"] if people else None

### Make a small table with name, height, mass
We’ll also convert height/mass to numbers where possible.

In [ ]:
rows = []
for p in people:
    h = pd.to_numeric(p.get("height"), errors="coerce")
    m = pd.to_numeric(p.get("mass").replace(",","") if isinstance(p.get("mass"), str) else p.get("mass"), errors="coerce")
    rows.append({"name": p.get("name"), "height_cm": h, "mass_kg": m})

swdf = pd.DataFrame(rows)
swdf.head(10)

### Simple plot: Top 8 tallest characters (by recorded height)

In [ ]:
tall = swdf.dropna(subset=["height_cm"]).sort_values("height_cm", ascending=False).head(8)
plt.bar(tall["name"], tall["height_cm"])
plt.title("Top 8 Tallest (SWAPI people)")
plt.xlabel("Character")
plt.ylabel("Height (cm)")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()
tall[["name","height_cm"]]

## Quick Reference
- GET JSON: `r = requests.get(url); data = r.json()`
- GET with params: `requests.get(url, params={"q":"python"})`
- POST JSON: `requests.post(url, json={"a":1})`
- Loop pages: increase `page` or follow `next` until it stops

## Wrap-Up
You just fetched real data from multiple APIs and visualized some of it. Next, try changing URLs, parameters, or fields to explore more!